In [1]:
#%pip install
#!pip3 install matplotlib
#!pip3 install scipy
#!pip3 instalsklearn
#!pip3 install joblib --upgrade
#!pip3 install fsspec
#!pip3 install s3fs 
#!pip3 install jmespath --upgrade
#!python3 -m jmespath install
#!pip3 install jmespath==0.9.4
#!conda install -c conda-forge jmespath

In [2]:
import s3fs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, kurtosis
from sklearn.covariance import LedoitWolf
import os

Se extraen los conjuntos de Datos de entrenamiento desde la carpeta results

In [3]:
Y_Train=pd.read_csv('s3://proyecto-integrador/results/Y_Entrenamiento.csv', header=0)
X_Train=pd.read_csv('s3://proyecto-integrador/results/X_Entrenamiento.csv', header=0)
Y_Train=Y_Train.drop([0])
X_Train=X_Train.drop([0])
Y_Train.iloc[:,0]=pd.to_datetime(Y_Train.iloc[:,0])
Y_Train.set_index('Fecha', inplace=True)
X_Train.iloc[:,0]=pd.to_datetime(X_Train.iloc[:,0])
X_Train.set_index('Fecha', inplace=True)

Desde la Zona Raw recuperamos los Datos originales. Tomaremos sólo la información correspondiente a los últimos 2 días.

In [4]:
df1=pd.read_csv('s3://proyecto-integrador/raw/DTF/DTF.csv', header=0)
df1=df1.drop([0])
df1=df1.iloc[:-4,:]
df1.iloc[:,0]=pd.to_datetime(df1.iloc[:,0])
#df1=df1.loc[np.logical_and(df1.iloc[:,0].dt.weekday!=5,df1.iloc[:,0].dt.weekday!=6)]
df1.set_index('Fecha', inplace=True)
df1.shape

(10358, 49)

In [5]:
df2=pd.read_csv('s3://proyecto-integrador/raw/TRM/TMR.csv', header=0)
df2=df2.drop([0])
df2=df2.iloc[:-4,:]
df2.iloc[:,0]=pd.to_datetime(df2.iloc[:,0])
#df2=df2.loc[np.logical_and(df2.iloc[:,0].dt.weekday!=5,df2.iloc[:,0].dt.weekday!=6)]
df2.set_index('Fecha', inplace=True)
df2.shape

(2503, 92)

In [6]:
dfs = pd.merge(df2,df1,on='Fecha',how='outer')
dfs=dfs.sort_index(axis=0)

In [7]:
dfs.iloc[0,:]=dfs.iloc[0,:].fillna(0)
dfs=dfs.fillna(method='ffill')
dfs=dfs.iloc[0:-1,:]
X_Prediccion = dfs.iloc[-2:]
X_Prediccion = X_Prediccion[X_Train.columns]

In [8]:
X_Pred_Final = X_Prediccion.copy()
X_Pred_Final.iloc[0,:] = (X_Prediccion.iloc[1,:]/X_Prediccion.iloc[0,:])-1 
X_Pred_Final = X_Pred_Final.iloc[0]

In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='elasticnet',multi_class='auto',solver='saga',max_iter=1000, random_state=0, warm_start=True,fit_intercept=True, l1_ratio=0.5)
model.fit(X_Train, Y_Train)

/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(l1_ratio=0.5, max_iter=1000, penalty='elasticnet',
                   random_state=0, solver='saga', warm_start=True)

In [10]:
Prediccion = model.predict(np.array(X_Pred_Final.fillna(0)).reshape(1,-1))

/home/emr-notebook/.local/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [11]:
Prediccion=3
if Prediccion == 1:
    PrediccionText='La tendencia del día de hoy será a la baja'
elif Prediccion == 2:
    PrediccionText='La tendencia del día de hoy será a la estable'
elif Prediccion == 3:
    PrediccionText='La tendencia del día de hoy será a la alza'

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_Train,model.predict(X_Train))

array([[ 30, 372,   0],
       [ 42, 369,   0],
       [ 39, 342,   2]])

In [13]:
PrediccionText

'La tendencia del día de hoy será a la alza'

In [15]:
pd.DataFrame([PrediccionText]).to_csv('s3://proyecto-integrador/results/ResultadoModelo.csv')